# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [4]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott', 'mayor':'booboo'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich','mayor':'bababs'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [5]:
json_normalize(data,'counties',[['info','governor'],['info','mayor']])

,name,population,info.governor,info.mayor
0,Dade,12345,Rick Scott,booboo
1,Broward,40000,Rick Scott,booboo
2,Palm Beach,60000,Rick Scott,booboo
3,Summit,1234,John Kasich,bababs
4,Cuyahoga,1337,John Kasich,bababs


In [6]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,state,shortname
0,Dade,12345,Rick Scott,Florida,FL
1,Broward,40000,Rick Scott,Florida,FL
2,Palm Beach,60000,Rick Scott,Florida,FL
3,Summit,1234,John Kasich,Ohio,OH
4,Cuyahoga,1337,John Kasich,Ohio,OH


In [7]:
json_normalize(data, 'counties', ['state'])

,name,population,state
0,Dade,12345,Florida
1,Broward,40000,Florida
2,Palm Beach,60000,Florida
3,Summit,1234,Ohio
4,Cuyahoga,1337,Ohio


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [9]:
# load json as string
json.load((open('datasets/world_bank_projects_less.json')))

[{u'_id': {u'$oid': u'52b213b38594d8a2be17c780'},
  u'approvalfy': 1999,
  u'board_approval_month': u'November',
  u'boardapprovaldate': u'2013-11-12T00:00:00Z',
  u'borrower': u'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  u'closingdate': u'2018-07-07T00:00:00Z',
  u'country_namecode': u'Federal Democratic Republic of Ethiopia!$!ET',
  u'countrycode': u'ET',
  u'countryname': u'Federal Democratic Republic of Ethiopia',
  u'countryshortname': u'Ethiopia',
  u'docty': u'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  u'envassesmentcategorycode': u'C',
  u'grantamt': 0,
  u'ibrdcommamt': 0,
  u'id': u'P129828',
  u'idacommamt': 130000000,
  u'impagency': u'MINISTRY OF EDUCATION',
  u'lendinginstr': u'Investment Project Financing',
  u'lendinginstrtype': u'IN',
  u'lendprojectcost': 550000000,
  u'majorsector_percent': [{u'Name': u'Education', u'Percent': 46},
   {u'Name': u'Education', u'Percent': 26},
   {u'Name': u'Public Administration, Law, and Ju

****
## JSON exercise


Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

In [19]:
# 1. find the 10 countries with the most projects
df = pd.read_json('datasets/world_bank_projects.json')
df.groupby(['countrycode','countryname']).size().sort_values(ascending=False).head(10)

countrycode  countryname                    
CN           People's Republic of China         19
ID           Republic of Indonesia              19
VN           Socialist Republic of Vietnam      17
IN           Republic of India                  16
RY           Republic of Yemen                  13
NP           Nepal                              12
BD           People's Republic of Bangladesh    12
MA           Kingdom of Morocco                 12
MZ           Republic of Mozambique             11
3A           Africa                             11
dtype: int64

In [20]:
#calculating the above differently as a double check 
df.groupby('countryname')['project_name'].count().sort_values(ascending=False).head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: project_name, dtype: int64

In [21]:
#2. Find the top 10 major project themes (using column 'mjtheme_namecode')
#df.groupby('mjtheme_namecode').
list1=[]
list2=[]
for x in range(len(df['mjtheme_namecode'])):
    for y in range(len(df['mjtheme_namecode'][x])):
        list1.append(df['mjtheme_namecode'][x][y][ u'name'])
        list2.append(df['mjtheme_namecode'][x][y][ u'code'])
projects=pd.DataFrame({'project themes': list1,'project codes': list2})
projects.groupby('project themes').size().sort_values(ascending=False)

project themes
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
dtype: int64

In [22]:
#3.  Create a dataframe with the missing names filled in.  
# The below is a dataframe excluding project themes and containing a count of non-null 
# project themes.  Of course, this count excludes project themes which are null when
# they should be!  So this isn't optimal!
new_df1=projects[projects['project themes']!=''].groupby(['project codes','project themes'])\
.size().sort_values(ascending=False)
new_df=pd.DataFrame(new_df1)
new_df.columns=['size']
new_df.index.names=['codes','names']
new_df

,,size
codes,names,
11,Environment and natural resources management,223
10,Rural development,202
8,Human development,197
2,Public sector governance,184
6,Social protection and risk management,158
4,Financial and private sector development,130
7,Social dev/gender/inclusion,119
5,Trade and integration,72
9,Urban development,47


In [23]:
#3. To create the original dataframe with the null project theme names  filled in.
#First create a dictionary with (key= proj theme code, value= project theme name) key:value pairs
s=new_df['size']
dict={}
for i in range(len(new_df.index.values)):
    dict[new_df.index.values[i][0]]=new_df.index.values[i][1]
dict

{u'1': u'Economic management',
 u'10': u'Rural development',
 u'11': u'Environment and natural resources management',
 u'2': u'Public sector governance',
 u'3': u'Rule of law',
 u'4': u'Financial and private sector development',
 u'5': u'Trade and integration',
 u'6': u'Social protection and risk management',
 u'7': u'Social dev/gender/inclusion',
 u'8': u'Human development',
 u'9': u'Urban development'}

In [24]:
#Create the non-null dataframe
import copy
df2=copy.deepcopy(df)
for i in range(len(df2['mjtheme_namecode'])):
    for j in range(len(df2['mjtheme_namecode'][i])):
        df2['mjtheme_namecode'][i][j][u'name']=dict[df2['mjtheme_namecode'][i][j][u'code']]

In [25]:
#check that the null project theme names are filled in
df['mjtheme_namecode'][0], df2['mjtheme_namecode'][0]

([{u'code': u'8', u'name': u'Human development'},
  {u'code': u'11', u'name': u''}],
 [{u'code': u'8', u'name': u'Human development'},
  {u'code': u'11', u'name': u'Environment and natural resources management'}])

In [26]:
#a more comprehensive check that there are no null project theme values in this series!
a=[]
b=[]
for x in range(len(df2['mjtheme_namecode'])):
    for y in range(len(df2['mjtheme_namecode'][x])):
        a.append(df2['mjtheme_namecode'][x][y][ u'name'])
        b.append(df2['mjtheme_namecode'][x][y][ u'code'])
projects2=pd.DataFrame({'project themes': a,'project codes': b})
projects2.groupby('project themes').size().sort_values(ascending=False)


project themes
Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Rule of law                                      15
dtype: int64